In [ ]:
from spark_utils import get_spark_session


spark = get_spark_session()
# run transform_latest.py if this don't exist
test_data = spark.read.parquet('data_transformed/amex-default-prediction/test_data_latest')
train_data = spark.read.parquet('data_transformed/amex-default-prediction/train_data_latest')
# run format_data.py if these don't exist
train_labels = spark.read.parquet('data/amex-default-prediction/train_labels')
sample_submission = spark.read.parquet('data/amex-default-prediction/sample_submission')

In [ ]:
%%time
from format_data import CATEGORICAL_VARIABLES
from encoder import CategoricalToIntegerEncoders
from format_data import TARGET_VARIABLE, DATE_VARIABLES, ID_VARIABLES


encs = CategoricalToIntegerEncoders(columns=CATEGORICAL_VARIABLES).fit(train_data)

train_pdf = train_data.join(train_labels, on='customer_ID', how='inner')
train_pdf = encs.transform(spark=spark, df=train_pdf).toPandas()

test_pdf = encs.transform(spark=spark, df=test_data).toPandas()

feature_columns = [
    c for c in train_pdf.columns 
    if c not in [TARGET_VARIABLE,] + ID_VARIABLES + list(DATE_VARIABLES.keys())
]
print('feature_columns\n', ', '.join(feature_columns))

In [ ]:
from sklearn.model_selection import train_test_split

X = train_pdf[feature_columns]
X_test = test_pdf[feature_columns]
y = train_pdf[TARGET_VARIABLE]
print('y.unique()', y.unique())

X_train, X_valid, y_train, y_valid = train_test_split(X, y)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
def get_weights(y, negative_label_weight: float = 20.):
    # default is thus 20. because:
    # Note that the negative class has been subsampled for this dataset at 5%, and thus receives a 20x weighting in the scoring metric.
    return y.apply(lambda x: negative_label_weight if x == 0. else 1.)

In [ ]:
%%time
import mlflow
from lightgbm import LGBMClassifier
from evaluation import evaluate
import pandas as pd
from format_data import PREDICTION_VARIABLE
from tempfile import TemporaryDirectory
import os
import numpy as np


for negative_label_weight_train in np.linspace(0.01, 20., num=10, endpoint=False):
    # mlflow.lightgbm.autolog()
    experiment_id = mlflow.get_experiment_by_name('use_latest_tune_label_weight.ipynb').experiment_id
    with mlflow.start_run(experiment_id=experiment_id) as run:
        run_id = run.info.run_id
        print(f'run_id: {run_id}')

        w_train = get_weights(y_train, negative_label_weight=negative_label_weight_train)
        mlflow.log_param('negative_label_weight_train', negative_label_weight_train)

        m = LGBMClassifier().fit(
            X=X_train, y=y_train, sample_weight=w_train,
            categorical_feature=encs.columns_encoded,
        )

        score_train = m.score(X=X_train, y=y_train, sample_weight=get_weights(y_train))
        mlflow.log_metric('score_train', score_train)
        score_valid = m.score(X=X_valid, y=y_valid, sample_weight=get_weights(y_valid))
        mlflow.log_metric('score_valid', score_valid)
        score_amex_train = evaluate(X_train, y_train, m=m)
        mlflow.log_metric('score_amex_train', score_amex_train)
        score_amex_valid = evaluate(X_valid, y_valid, m=m)
        mlflow.log_metric('score_amex_valid', score_amex_valid)

        pred_df = pd.DataFrame({
            PREDICTION_VARIABLE: m.predict_proba(X_test)[:, 1],
            'customer_ID': test_pdf['customer_ID'],
        })
        pred_and_sample_joined_counts = (
            spark
            .createDataFrame(pred_df)
            .join(sample_submission, on='customer_ID', how='inner')
            .count()
        )
        assert pred_and_sample_joined_counts == len(pred_df), \
            f'''These should be identical:
            sample_submission has {sample_submission.count()} rows,
            pred_and_sample_joined_counts is {pred_and_sample_joined_counts},
            pred_df has {len(pred_df)} rows
            '''
        with TemporaryDirectory() as p:
            p = os.path.join(p, 'submission.csv')
            pred_df.to_csv(p, header=True, index=False)
            mlflow.log_artifact(local_path=p)
